In [31]:
from imutils import paths
import numpy as np
import argparse
import os
import cv2
import time

In [32]:
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", default =r'dataset1', help="dataset")
ap.add_argument("-f", "--face", type=str, default="face_detector", help="face detector model")
ap.add_argument("-c", "--confidence", type=float, default=0.8, help="minimum probability")
args = vars(ap.parse_args(args=[]))

- 얼굴을 디텍션할 모델 로드

In [33]:
prototxtPath = os.path.sep.join([args["face"], "deploy.prototxt"])
weightsPath = os.path.sep.join([args["face"], "res10_300x300_ssd_iter_140000.caffemodel"])
net = cv2.dnn.readNet(prototxtPath, weightsPath)

- 얼굴을 CROP할 이미지 경로 위치

In [34]:
imagePaths = list(paths.list_images(args["dataset"]))

- CROP한 이미지들이 저장될 경로 폴더

In [35]:
preprocessed_dir = "mask_data_preprocessed"

In [36]:
time_list= []

In [40]:
for index, imagePath in enumerate(imagePaths):
    # 폴더 명 with_mask / without_mask 는 라벨
    label = imagePath.split(os.path.sep)[-2]

    label_dir = os.path.join(preprocessed_dir, label)
    os.makedirs(label_dir, exist_ok=True)
    # 이미지 읽고
    img = cv2.imread(imagePath)
    if img is None:
        # 못읽으면 넘어감
        print(img)
        continue
    # RGB로 변경
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    (h, w) = image.shape[:2]

    # BLOB 생성
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),
                                 (104.0, 177.0, 123.0))

    print("[INFO] computing face detections...")
    start = time.time()
    # 네트워크 input에 BLOB 설정
    net.setInput(blob)
    # 디텍션 실행
    detections = net.forward()

    # 디텍션 된 위치 잘라내기
    cnt = 0
    boxes = [detections[0, 0, i, 3:7] * np.array([w, h, w, h]) 
             for i in range(detections.shape[2]) if detections[0, 0, i, 2] > args["confidence"]]
    for box in boxes:
        (startX, startY, endX, endY) = box.astype("int")
        (startX, startY) = (max(0, startX), max(0, startY))
        (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

        face = image[startY:endY, startX:endX]
        if len(face) == 0:
            continue
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        # 이미지 저장
        cv2.imwrite(label_dir + "/" + str(index) + "_" + str(cnt) + ".png", face)
        cnt += 1

    second = time.time()
    print(second-start)
    time_list.append(second-start)
print('complete')

[INFO] computing face detections...
0.1472456455230713
[INFO] computing face detections...
0.1539292335510254
[INFO] computing face detections...
0.15624523162841797
[INFO] computing face detections...
0.15968084335327148
[INFO] computing face detections...
0.17188119888305664
[INFO] computing face detections...
0.16054749488830566
[INFO] computing face detections...
0.15063047409057617
[INFO] computing face detections...
0.21603751182556152
[INFO] computing face detections...
0.286592960357666
[INFO] computing face detections...
0.18440508842468262
[INFO] computing face detections...
0.14488792419433594
[INFO] computing face detections...
0.15208673477172852
[INFO] computing face detections...
0.17600083351135254
[INFO] computing face detections...
0.17299580574035645
[INFO] computing face detections...
0.1640021800994873
[INFO] computing face detections...
0.17600083351135254
[INFO] computing face detections...


KeyboardInterrupt: 

In [41]:
print(sum(time_list)/len(time_list))

0.17469076549305634
